In [163]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
from  backtest import Backtester,  get_ohlc_history, create_price_fig, evaluate_backtest

In [164]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [165]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's').dt.tz_localize('UTC')
    return df

In [166]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,6,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
#df.set_index("time", inplace=True)
#df = df.between_time("13:00", "16:00")
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-06-02 00:00:00+00:00,1.13463,1.13463,1.13398,1.13398,2,70,0
1,2025-06-02 00:01:00+00:00,1.13398,1.13398,1.13398,1.13398,1,200,0
2,2025-06-02 00:02:00+00:00,1.13398,1.13398,1.13398,1.13398,1,200,0
3,2025-06-02 00:03:00+00:00,1.13398,1.13398,1.13390,1.13390,2,200,0
4,2025-06-02 00:04:00+00:00,1.13390,1.13390,1.13382,1.13382,2,200,0
...,...,...,...,...,...,...,...,...
50344,2025-07-18 23:52:00+00:00,1.16208,1.16213,1.16188,1.16194,44,7,0
50345,2025-07-18 23:53:00+00:00,1.16194,1.16219,1.16194,1.16218,49,7,0
50346,2025-07-18 23:54:00+00:00,1.16220,1.16220,1.16213,1.16220,16,12,0
50347,2025-07-18 23:55:00+00:00,1.16220,1.16223,1.16214,1.16223,14,10,0


In [178]:
# === Calculate indicators ===
df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5

# === Signal Generation ===
def generate_signals(df):
    signals = []
    for i in range(14, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]

        cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
        cond2 = row["spread"] < 0.0002
        cond3 = row["volume_spike"]
        cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

        if cond1 and cond2 and cond3 and cond4: 
            signals.append(i)
    return signals

signals = generate_signals(df)

# === Backtest ===
results = []
rr = 3  # risk-reward ratio

for i in signals:
    entry_price = df.iloc[i]["close"]
    sl = entry_price - 0.5 * df.iloc[i]["atr"]
    tp = entry_price + rr * (entry_price - sl)

    for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
        high = df.iloc[j]["high"]
        low = df.iloc[j]["low"]
        if low <= sl:
            results.append(("loss", entry_price, sl))
            break
        elif high >= tp:
            results.append(("win", entry_price, tp))
            break
    else:
        # time-based exit
        close_exit = df.iloc[i+60]["close"]
        if close_exit > entry_price:
            results.append(("win", entry_price, close_exit))
        else:
            results.append(("loss", entry_price, close_exit))
# === Metrics ===
wins = [r for r in results if r[0] == "win"]
losses = [r for r in results if r[0] == "loss"]
win_rate = len(wins) / len(results)
r_r_ratio = np.mean([abs(w[2]-w[1]) for w in wins]) / np.mean([abs(l[2]-l[1]) for l in losses])


entry_price = df.iloc[i]["close"]
df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
df['tp'] = entry_price + rr * (entry_price - sl)

print(f"Total trades: {len(results)}")
print(f"Win rate: {win_rate:.2%}")
print(f"Risk-Reward Ratio: {r_r_ratio:.2f}")
print(results)


Total trades: 33
Win rate: 45.45%
Risk-Reward Ratio: 3.81
[('win', np.float64(1.13805), np.float64(1.13831)), ('loss', np.float64(1.13723), np.float64(1.13709)), ('loss', np.float64(1.13724), np.float64(1.13697)), ('win', np.float64(1.13713), np.float64(1.1376999999999997)), ('win', np.float64(1.13756), np.float64(1.1377999999999995)), ('loss', np.float64(1.13769), np.float64(1.137555)), ('loss', np.float64(1.13769), np.float64(1.1375450000000003)), ('win', np.float64(1.13784), np.float64(1.1383949999999996)), ('loss', np.float64(1.13837), np.float64(1.138175)), ('win', np.float64(1.13752), np.float64(1.13843)), ('loss', np.float64(1.13821), np.float64(1.13789)), ('loss', np.float64(1.13776), np.float64(1.137385)), ('loss', np.float64(1.13791), np.float64(1.1377549999999998)), ('loss', np.float64(1.13729), np.float64(1.13701)), ('loss', np.float64(1.13676), np.float64(1.13659)), ('win', np.float64(1.13587), np.float64(1.13743)), ('loss', np.float64(1.13644), np.float64(1.13598499999999

In [168]:
prev = df.iloc[i-1]
def entring_stoploss_profit(row, prev) :
    hour = row['time'].hour
    if row["close"] < row["vwap"] - 2 * row["atr"] and row["spread"] < 0.0002 and row["volume_spike"] and row["close"] > row["open"] and prev["close"] < prev["open"] :#and(hour > 13 and hour < 15):
        return "buy"
    #elif signals :
        #return "sell"
df["signal"] = df.apply(entring_stoploss_profit, axis=1, prev = prev)
df.copy().dropna()

,time,open,high,low,close,tick_volume,spread,real_volume,atr,vwap,volume_spike,sl,tp,signal
2293,2025-06-03 14:14:00+00:00,1.13921,1.13947,1.13920,1.13939,74,0,0,0.00071,1.141281,True,1.15707,1.15855,buy
2328,2025-06-03 14:49:00+00:00,1.13936,1.13952,1.13936,1.13945,64,0,0,0.00060,1.141245,True,1.15707,1.15855,buy
2373,2025-06-03 15:34:00+00:00,1.13783,1.13825,1.13772,1.13805,95,0,0,0.00105,1.141186,True,1.15707,1.15855,buy
2397,2025-06-03 15:58:00+00:00,1.13810,1.13822,1.13804,1.13814,89,0,0,0.00052,1.141148,True,1.15707,1.15855,buy
2555,2025-06-03 18:36:00+00:00,1.13754,1.13796,1.13750,1.13796,98,0,0,0.00059,1.140868,True,1.15707,1.15855,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19934,2025-06-19 20:31:00+00:00,1.14627,1.14631,1.14627,1.14631,22,0,0,0.00031,1.147093,True,1.15707,1.15855,buy
48359,2025-07-17 14:46:00+00:00,1.15755,1.15765,1.15752,1.15764,44,0,0,0.00048,1.159099,True,1.15707,1.15855,buy
48371,2025-07-17 14:58:00+00:00,1.15731,1.15744,1.15722,1.15744,73,0,0,0.00074,1.159099,True,1.15707,1.15855,buy
48372,2025-07-17 14:59:00+00:00,1.15744,1.15758,1.15740,1.15745,90,0,0,0.00071,1.159099,True,1.15707,1.15855,buy


In [ ]:

# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' :#and not num_open_trades:
        volume = 400000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        volume = 400000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["low"] <= data["sl"] or data["high"] >= data["tp"]: 
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell'  :
            orders.close_trade(trade)

In [170]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,400000,2025-06-03 14:14:00+00:00,1.13921,2025-06-03 14:15:00+00:00,1.13939,0,0,{},72.0,-28.0,44.0,44.0,10044.0
1,closed,EURUSD,buy,400000,2025-06-03 14:49:00+00:00,1.13936,2025-06-03 14:50:00+00:00,1.13945,0,0,{},36.0,-28.0,8.0,52.0,10052.0
2,closed,EURUSD,buy,400000,2025-06-03 15:34:00+00:00,1.13783,2025-06-03 15:35:00+00:00,1.13805,0,0,{},88.0,-28.0,60.0,112.0,10112.0
3,closed,EURUSD,buy,400000,2025-06-03 15:58:00+00:00,1.1381,2025-06-03 15:59:00+00:00,1.13814,0,0,{},16.0,-28.0,-12.0,100.0,10100.0
4,closed,EURUSD,buy,400000,2025-06-03 18:36:00+00:00,1.13754,2025-06-03 18:37:00+00:00,1.13796,0,0,{},168.0,-28.0,140.0,240.0,10240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,closed,EURUSD,buy,400000,2025-06-19 20:31:00+00:00,1.14627,2025-06-19 20:32:00+00:00,1.1463,0,0,{},12.0,-28.0,-16.0,1096.0,11096.0
77,closed,EURUSD,buy,400000,2025-07-17 14:46:00+00:00,1.15755,2025-07-17 14:50:00+00:00,1.15725,0,0,{},-120.0,-28.0,-148.0,948.0,10948.0
78,closed,EURUSD,buy,400000,2025-07-17 14:58:00+00:00,1.15731,2025-07-17 15:17:00+00:00,1.15831,0,0,{},400.0,-28.0,372.0,1320.0,11320.0
79,closed,EURUSD,buy,400000,2025-07-17 14:59:00+00:00,1.15744,2025-07-17 15:18:00+00:00,1.15857,0,0,{},452.0,-28.0,424.0,1744.0,11744.0


In [174]:
mm = bt.plot_balance()
display(mm)

In [172]:
evaluate_backtest(bt.trades)

biggest_profit: 460.0
daily_drawdown: -120.0
max_drawown: -236.0
max_drawown1: 44.0
avg_win: 57.05
avg_loss: -120.0
count_profit_trades: 80
count_loss_trades: 1
Win Rate1: 98.77%
rrr: 0.48


,order_type,profit
0,buy,4444.0


c:\Users\hp\python_finance_market_projects\backtest.py:376: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit_net,profit_cumulative,balance,current_max,drawdown,drawdown1,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,400000,2025-06-03 14:14:00+00:00,1.13921,2025-06-03 14:15:00+00:00,1.13939,0,0,...,44.0,44.0,10044.0,44.0,0.0,44.0,1,14,2025-06,2025
1,closed,EURUSD,buy,400000,2025-06-03 14:49:00+00:00,1.13936,2025-06-03 14:50:00+00:00,1.13945,0,0,...,8.0,52.0,10052.0,52.0,0.0,44.0,1,14,2025-06,2025
2,closed,EURUSD,buy,400000,2025-06-03 15:34:00+00:00,1.13783,2025-06-03 15:35:00+00:00,1.13805,0,0,...,60.0,112.0,10112.0,112.0,0.0,44.0,1,15,2025-06,2025
3,closed,EURUSD,buy,400000,2025-06-03 15:58:00+00:00,1.1381,2025-06-03 15:59:00+00:00,1.13814,0,0,...,-12.0,100.0,10100.0,112.0,-12.0,44.0,1,15,2025-06,2025
4,closed,EURUSD,buy,400000,2025-06-03 18:36:00+00:00,1.13754,2025-06-03 18:37:00+00:00,1.13796,0,0,...,140.0,240.0,10240.0,240.0,0.0,44.0,1,18,2025-06,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,closed,EURUSD,buy,400000,2025-06-19 20:31:00+00:00,1.14627,2025-06-19 20:32:00+00:00,1.1463,0,0,...,-16.0,1096.0,11096.0,1184.0,-88.0,44.0,3,20,2025-06,2025
77,closed,EURUSD,buy,400000,2025-07-17 14:46:00+00:00,1.15755,2025-07-17 14:50:00+00:00,1.15725,0,0,...,-148.0,948.0,10948.0,1184.0,-236.0,44.0,3,14,2025-07,2025
78,closed,EURUSD,buy,400000,2025-07-17 14:58:00+00:00,1.15731,2025-07-17 15:17:00+00:00,1.15831,0,0,...,372.0,1320.0,11320.0,1320.0,0.0,44.0,3,14,2025-07,2025
79,closed,EURUSD,buy,400000,2025-07-17 14:59:00+00:00,1.15744,2025-07-17 15:18:00+00:00,1.15857,0,0,...,424.0,1744.0,11744.0,1744.0,0.0,44.0,3,14,2025-07,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.